Remove useless neurons

In [10]:
import numpy as np
weights = np.load('weights.npy')
hidden_outputs = np.load('hidden_outputs.npy')
states = np.load('observations.npy')
actions = np.load('actions.npy')
print(weights.shape)
index = states[:,0]==21
states = states[index]
hidden_outputs = hidden_outputs[index]
actions = actions[index]
print(states[0])
print(hidden_outputs[0])
print(hidden_outputs[0].reshape(1,64)@weights)
print(actions[0])


(64, 2)
[21  9  1]
[ -0.3050744   -3.3005354   -0.19211519  -0.55337304  -0.6725486
  -0.48471168  -0.81506175  -9.164504    -0.97856236   0.6999247
  -1.8707147   -4.10077     -0.99064034  -0.7340159   -1.4050695
  -0.24971282  -1.3867046   -0.61150247  -2.8810499   -7.1059012
   2.008685    -5.9896917   -0.5528022    0.21797323  -5.7041044
  -0.3118731   -0.20818149  -3.7404547   -9.375502    -8.10238
  -1.0194423   -0.5091292   -6.633438    -9.479743    -7.7186375
   1.7107706   -0.52075315  -5.111542    -0.50309795  -0.75730795
  -4.242246    -0.44974715  -0.84749633  -0.94891894  -0.26672843
  -0.5260327   -1.7301596   -1.975936     2.5473194  -10.748297
  -1.6153109   -3.9273074   -1.0840628   -4.644953    -4.5544505
  -3.9086409   -1.4812176   -8.031258    -0.54643714  -0.66253084
  -0.62048554  -0.31940624  -1.1657099  -11.956685  ]
[[26.84132  13.382536]]
0


In [4]:
import os
import torch
import numpy as np
import gymnasium as gym
from tianshou.data import Collector, VectorReplayBuffer, Batch
from tianshou.utils.net.common import Net
from gymnasium.wrappers import TransformObservation
from tianshou.env import SubprocVectorEnv
from tianshou.policy import DQNPolicy
import pprint


def make_env():
    env = gym.make("Blackjack-v1", natural=False, sab=False)
    env = TransformObservation(env, lambda obs: np.array(obs), observation_space=None)
    return env


DEVICE = "cuda:0"
env = make_env()
state_shape = 3
action_shape = env.action_space.shape or env.action_space.n

q_net = Net(state_shape, action_shape, hidden_sizes=[64] * 2, device=DEVICE).to(DEVICE)
optim = torch.optim.Adam(q_net.parameters(), lr=2.5e-4)
policy = DQNPolicy(
    model=q_net,
    optim=optim,
    action_space=env.action_space,
    discount_factor=0.99,
    estimation_step=3,
    target_update_freq=500,
).to(DEVICE)
policy.eval()
checkpoint_path = "/root/gym/rl_compexp/save/Blackjack-DQN64/dqn_best.pth"
policy.load_state_dict(torch.load(checkpoint_path,map_location=DEVICE))
hidden_outputs = []
def hook_fn(module, input, output):
    # Assuming output is a tensor, detach and move to CPU
    hidden_outputs.append(output.detach().cpu().numpy())


policy.model.model.model[2].register_forward_hook(hook_fn)

weight = policy.model.model.model[4].weight.detach().cpu().t().numpy()
state = np.array([21,9,1])
print(state.shape)
action = policy(Batch(obs=state.reshape(1, -1), info=""))


(3,)


In [10]:
print(policy.model.model.model[4].weight[:,9])
print(hidden_outputs[0][0][9])
print(action)

tensor([-0.3466,  0.1134], device='cuda:0', grad_fn=<SelectBackward0>)
0.69992423
Batch(
    logits: tensor([[0.9306, 0.1792]], device='cuda:0', grad_fn=<AddmmBackward0>),
    act: array([0]),
    state: None,
)


In [1]:
import pandas as pd

mean_iou = {}
i = 5
res = pd.read_csv(f"result_{i}.csv")
res = res[res["iou"] > 0]
res = res.sort_values(by="iou", ascending=False)
mean_iou[i] = res["iou"].mean()
res.to_csv(f"result_{i}_parsed.csv", index=False)

find 4 outputs top connections

In [2]:
import pandas as pd

i = 5
N = 10
csv = pd.read_csv(f"result_{i}_parsed.csv")
output_file = f"output_{i}.txt"
neurons = []


def find_unique_elements(arr1, arr2):
    set1, set2,  = set(arr1), set(arr2)
    unique1 = set1 - set2 
    unique2 = set2 - set1 


    return unique1, unique2


for label in ["w_stick","w_hit"]:
    sorted_csv = csv.sort_values(by=label, ascending=False)
    top_neurons = sorted_csv.head(N)["neuron"].tolist()
    neurons.append(top_neurons)

unique_elements = find_unique_elements(neurons[0], neurons[1])
print(unique_elements, "\n")

with open(output_file, "w") as f:
    for i, label in enumerate(["w_stick", "w_hit"]):
        csv = csv.sort_values(by=label, ascending=False)
        top_neurons = csv.head(N)[["neuron", label]].to_records(index=False)
        print(f"\n============\n{label} top {N}:", file=f)
        for neuron, value in top_neurons:
            if neuron in unique_elements[i]:
                print(f"* {neuron} {value}", file=f)  # Special mark for unique elements
                print(f"* {neuron} {value}")
            else:
                print(f"  {neuron} {value}", file=f)
                print(f"  {neuron} {value}")

({48, 37}, {9, 28}) 

  11 0.5003815
  35 0.3611781
  20 0.3492526
* 48 0.15106824
  59 0.13365164
  57 0.09165179
  23 0.047311936
* 37 0.01940344
  2 -0.20769313
  34 -0.24242833
  57 0.90779394
  23 0.25493503
  20 0.2245203
  59 0.18274003
  35 0.13949797
* 9 0.11335883
  11 -0.020975383
  2 -0.04273699
  34 -0.062015936
* 28 -0.11444995


find act & inp pair

In [2]:
import numpy as np

act = np.load(f"../save/LunarLander-{NET_NAME}/actions.npy")
obs = np.load("../save/lunar.npy")

act_list = []
act_idx_list = []
max_indices = np.argmax(act, axis=1)
print(max_indices)
print(max_indices.shape)
for i in range(4):
    max_row_index = np.where(max_indices == i)[0]
    print(max_row_index.shape)
    # max_row = act[max_row_index]
    act_idx_list.append(max_row_index)
    act_list.append(obs[max_row_index])

# np.save(f'../save/LunarLander-{NET_NAME}/max_indices.npy', max_indices)

[3 3 3 ... 1 1 1]
(10000,)
(1715,)
(2127,)
(2696,)
(3462,)


## analyze specific neuron(DQN)

In [26]:
from analyze import *

ckpt_path = "../save/ckpt-mlp1024-260.pth"
data_path = "../save/lunar.npy"
print("Load model and dataset")
model, dataset = load_for_analysis(ckpt_path, data_path)
weights = model.fc3.weight.t().detach().cpu().numpy()

print("Extract features")
inputs, features, outputs = extract_feature(model, dataset)
outputs = np.array(outputs)
if not os.path.exists(settings.RESULT):
    # 如果不存在，创建路径
    os.makedirs(settings.RESULT)
np.save(os.path.join(settings.RESULT, f"actions.npy"), outputs)
print("Computing quantiles")
activations = quantile_features(features)
print("Generating atomic masks")
feat_masks = gen_feat_mask(inputs)
print(feat_masks.shape)

Load model and dataset
Extract features


100%|██████████| 157/157 [00:00<00:00, 419.39it/s]


Computing quantiles
Generating atomic masks
(10000, 16)


[x, y, Vx, Vy, angle, angular_v, l, r]

y_near_ground: <0.4

Vy_low: -0.5 ~ 0

In [39]:
def intersect(x: np.ndarray, y: np.ndarray):
    return np.array(list(set(x).intersection(set(y))))


vy_low = np.where(feat_masks[:, 7] > 0)[0]
y_near_ground = np.where(feat_masks[:, 4] > 0)[0]

vy_low_and_y_near_ground = intersect(vy_low, y_near_ground)

neuron_act = np.where(activations[:, 630] > 0)[0]
res = intersect(neuron_act, act_idx_list[0])
res = intersect(res, vy_low_and_y_near_ground)
print(res.shape)
print(act[res[10]])
print(obs[res[10]])
print(features[res[10]][630])
hook = HookLayer()
hook.hook_layer(layer=model.act2)
obs_attk = obs[res[10]]
obs_attk[2] = 1.0
print(obs_attk)
act_attk = model(torch.tensor(obs_attk, dtype=torch.float32).to(settings.DEVICE))
print(hook.features_blobs)

(1237,)
[54.300285 52.586964 50.26787  52.025227]
[ 0.30821085  0.35956365  1.         -0.44264442  0.39747119 -0.08431965
  0.          0.        ]
0.63623697
[ 0.30821085  0.35956365  1.         -0.44264442  0.39747119 -0.08431965
  0.          0.        ]
[array([0., 0., 0., ..., 0., 0., 0.], dtype=float32)]


In [40]:
print(hook.features_blobs[0][630])
print(act_attk)

0.0
tensor([88.7608, 86.5204, 93.6968, 86.7465], device='cuda:3',
       grad_fn=<ViewBackward0>)


## analyze attack (PPO)


In [3]:
from PPO import get_PPO
import settings
from analyze import *

ckpt_path = "../save/LunarLander-PPO1024/ppo.pth"
data_path = "../save/lunar.npy"
print("Load model and dataset")
inputs, features, outputs, weight, policy = get_PPO(ckpt_path, data_path)


if not os.path.exists(settings.RESULT):
    # 如果不存在，创建路径
    os.makedirs(settings.RESULT)
np.save(os.path.join(settings.RESULT, f"actions.npy"), outputs)
print("Computing quantiles")
activations = quantile_features(features)
print("Generating atomic masks")
feat_masks = gen_feat_mask(inputs)
print(feat_masks.shape)

Load model and dataset
Computing quantiles
Generating atomic masks
(10000, 16)


In [18]:
import tianshou


def intersect(x: np.ndarray, y: np.ndarray):
    return np.array(list(set(x).intersection(set(y))))


not_l_leg = np.where(feat_masks[:, 14] == 0)[0]
r_leg = np.where(feat_masks[:, 15] > 0)[0]
x_in_centor = np.where(feat_masks[:, 0] > 0)[0]

inter = intersect(not_l_leg, r_leg)
inter = intersect(x_in_centor, inter)

neuron_act = np.where(activations[:, 212] > 0)[0]
res = intersect(neuron_act, act_idx_list[2])
res = intersect(res, inter)
print(res.shape)
print(act[res[10]])
print(obs[res[10]])
print(features[res[10]][212])
hook = HookLayer()
hook.hook_layer(policy.actor.preprocess.model.model[2])
obs_attk = obs[res[10]]
obs_attk[0] = 0.9
obs_attk[6] = 1.0
obs_attk = obs_attk.reshape(1, -1)
print(obs_attk)
print(obs_attk.shape)
act_attk = policy(tianshou.data.Batch(obs=obs_attk, info=""))
print(hook.features_blobs[0][0][212])
print(act_attk)

(14,)
[0.28012165 0.15548752 0.48146957 0.0829213 ]
[ 0.9         0.01810535  1.         -0.2070954   0.21621577 -0.67224586
  1.          1.        ]
0.37163532
[[ 0.9         0.01810535  1.         -0.2070954   0.21621577 -0.67224586
   1.          1.        ]]
(1, 8)
-1.70284
Batch(
    logits: tensor([[1.2064e-01, 8.7784e-01, 1.1213e-03, 3.9262e-04]], device='cuda:3',
                   grad_fn=<SoftmaxBackward0>),
    act: tensor([1], device='cuda:3'),
    state: None,
    dist: Categorical(probs: torch.Size([1, 4])),
)
